In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import make_grid
import torchvision.transforms as transforms
from torch.utils.data import  DataLoader, Dataset

import numpy as np 
import matplotlib.pyplot as plt

import os
import sys
from PIL import Image

In [2]:
device = torch.device('cuda')

In [3]:
class Net(nn.Module):
    def __init__(self, num_channels):
        super(Net, self).__init__()

        self.num_channels = num_channels

        self.conv1 = nn.Conv2d(3, self.num_channels, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(self.num_channels, self.num_channels*2, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(self.num_channels*2, self.num_channels*4, 3, stride=1, padding=1)

        self.fc1 = nn.Linear(self.num_channels*4*8*8, self.num_channels*4)
        self.fc2 = nn.Linear(self.num_channels*4, 6)


    def forward(self, x):
        # Empieza 3x64x64
        nn.ReLU
        x = self.conv1(x) # num_channels x 64 x 64
        x = F.relu(F.max_pool2d(x, 2)) # num_channels x 32 x 32
        x = self.conv2(x) # num_channels*2 x 32 x 32
        x = F.relu(F.max_pool2d(x, 2)) # num_channels*2 x 16 x 16
        x = self.conv3(x) # num_channels*4 x 16 x 16
        x = F.relu(F.max_pool2d(x, 2)) # num_channels*4 x 8 x 8

        # Flatten
        x = x.view(-1, self.num_channels*4*8*8)

        # fc
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # log_softmax
        x = F.log_softmax(x, dim=1)

        return x

In [6]:
class SIGNSDataset(Dataset):
    def __init__(self, base_dir, split='train', transform=None):
        path = os.path.join(base_dir, '{}_signs'.format(split))
        files = os.listdir(path)

        self.filenames = [os.path.join(path,f) for f in files if f.endswith('.jpg')]
        self.targets = [int(f.split('/')[-1][0]) for f in self.filenames]

        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx])
        if self.transform:
            image = self.transform(image)

        return image, self.targets[idx]

In [7]:
base_dir = 'data/processed/64x64_SIGNS'

trainset = SIGNSDataset(base_dir, split='train', transform=transforms.ToTensor())
dataloader = DataLoader(trainset, batch_size=32)

In [8]:
net = Net(32).to(device)

loss_fn = nn.NLLLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

In [9]:
class RunningMetric():
    def __init__(self):
        self.S = 0
        self.N = 0

    def update(self, val, size):
        self.S += val
        self.N += size

    def __call__(self):
        return self.S / float(self.N)

In [10]:
num_epoch = 100

In [11]:
for epoch in range(num_epoch):
    print('Epoch {}/{}'.format(epoch+1, num_epoch))
    print('(', end='')

    running_loss = RunningMetric()
    running_acc = RunningMetric()

    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        _, preds = torch.max(outputs, 1)

        loss = loss_fn(outputs, targets)

        loss.backward() # Calculo de gradientes automaticamente
        optimizer.step() # Actualizacion de parametros

        batch_size = inputs.size()[0]

        running_loss.update(loss.item()*batch_size, batch_size)
        running_acc.update(torch.sum(preds == targets).float(), batch_size)
        print('=', end='')

    print(")Loss: {:.4f} Acc: {:.4f}".format(running_loss(), running_acc()) )

Epoch 1/100
(===========================)Loss: 1.7941 Acc: 0.1632
Epoch 2/100
(===========================)Loss: 1.7913 Acc: 0.1898
Epoch 3/100
(===========================)Loss: 1.7899 Acc: 0.2106
Epoch 4/100
(===========================)Loss: 1.7889 Acc: 0.2176
Epoch 5/100
(===========================)Loss: 1.7879 Acc: 0.2245
Epoch 6/100
(===========================)Loss: 1.7869 Acc: 0.2188
Epoch 7/100
(===========================)Loss: 1.7858 Acc: 0.2188
Epoch 8/100
(===========================)Loss: 1.7845 Acc: 0.2188
Epoch 9/100
(===========================)Loss: 1.7831 Acc: 0.2257
Epoch 10/100
(===========================)Loss: 1.7815 Acc: 0.2269
Epoch 11/100
(===========================)Loss: 1.7797 Acc: 0.2419
Epoch 12/100
(===========================)Loss: 1.7776 Acc: 0.2627
Epoch 13/100
(===========================)Loss: 1.7751 Acc: 0.2824
Epoch 14/100
(===========================)Loss: 1.7720 Acc: 0.3009
Epoch 15/100
(===========================)Loss: 1.7683 Acc: 0.3125
Epoc

In [12]:
transform = transforms.Compose(
    [
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [13]:
trainset = SIGNSDataset(base_dir, split='train', transform=transform)
trainloader = DataLoader(trainset, batch_size=32)

In [14]:
valset = SIGNSDataset(base_dir, split='val', transform=transform)
valloader = DataLoader(valset, batch_size=32)

In [15]:
testset = SIGNSDataset(base_dir, split='test', transform=transform)
testloader = DataLoader(testset, batch_size=32)

In [16]:
dataloaders = { 'train': trainloader,
                'test': testloader,
                'val': valloader
                }

In [17]:
class Net(nn.Module):
    def __init__(self, num_channels):
        super(Net, self).__init__()

        self.num_channels = num_channels

        self.conv1 = nn.Conv2d(3, self.num_channels, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(self.num_channels)
        self.conv2 = nn.Conv2d(self.num_channels, self.num_channels*2, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(self.num_channels*2)
        self.conv3 = nn.Conv2d(self.num_channels*2, self.num_channels*4, 3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(self.num_channels*4)

        self.fc1 = nn.Linear(self.num_channels*4*8*8, self.num_channels*4)
        self.fcbn1 = nn.BatchNorm1d(self.num_channels*4)
        self.fc2 = nn.Linear(self.num_channels*4, 6)


    def forward(self, x):
        # Empieza 3x64x64
        nn.ReLU
        x = self.bn1(self.conv1(x)) # num_channels x 64 x 64
        x = F.relu(F.max_pool2d(x, 2)) # num_channels x 32 x 32
        x = self.bn2(self.conv2(x)) # num_channels*2 x 32 x 32
        x = F.relu(F.max_pool2d(x, 2)) # num_channels*2 x 16 x 16
        x = self.bn3(self.conv3(x)) # num_channels*4 x 16 x 16
        x = F.relu(F.max_pool2d(x, 2)) # num_channels*4 x 8 x 8

        # Flatten
        x = x.view(-1, self.num_channels*4*8*8)

        # fc
        x = F.relu(self.fcbn1(self.fc1(x)))
        x = F.dropout(x, p=0.8, training=True)
        x = self.fc2(x)

        # log_softmax
        x = F.log_softmax(x, dim=1)

        return x

In [24]:
def train_and_evaluate(model, optimizer, loss_fn, dataloaders, device, num_epoch=10, lr=0.001):
    for g in optimizer.param_groups:
        g['lr'] = lr

    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch+1, num_epoch))
        print('(', end='')

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = RunningMetric()
            running_acc = RunningMetric()

            for inputs, targets in dataloaders[phase]:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    loss = loss_fn(outputs, targets)
                    if phase == 'train':
                        loss.backward() # Calculo de gradientes automaticamente
                        optimizer.step() # Actualizacion de parametros

                batch_size = inputs.size()[0]

                running_loss.update(loss.item()*batch_size, batch_size)
                running_acc.update(torch.sum(preds == targets).float(), batch_size)
                print('=', end='')

            print(")Loss: {:.4f} Acc: {:.4f}".format(running_loss(), running_acc()) )

In [20]:
import random

lrs = [10**(-random.randint(3, 7)) for _ in range(3)]
lrs

[0.001, 1e-05, 1e-07]

In [22]:
for lr in lrs:
    net = Net(32).to(device)

    loss_fn = nn.NLLLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

    print(lr)
    train_and_evaluate(net, optimizer, loss_fn, dataloaders, device, lr=lr)

0.001
Epoch 1/10
(===========================)Loss: 1.6320 Acc: 0.3542
=======)Loss: 1.9795 Acc: 0.2778
Epoch 2/10
(===========================)Loss: 1.1681 Acc: 0.5856
=======)Loss: 1.0904 Acc: 0.5880
Epoch 3/10
(===========================)Loss: 0.9525 Acc: 0.6632
=======)Loss: 0.9685 Acc: 0.6389
Epoch 4/10
(===========================)Loss: 0.8484 Acc: 0.7164
=======)Loss: 0.8809 Acc: 0.6944
Epoch 5/10
(===========================)Loss: 0.7594 Acc: 0.7616
=======)Loss: 0.7755 Acc: 0.7500
Epoch 6/10
(===========================)Loss: 0.6853 Acc: 0.7986
=======)Loss: 0.7059 Acc: 0.7593
Epoch 7/10
(===========================)Loss: 0.6013 Acc: 0.8252
=======)Loss: 0.6604 Acc: 0.7778
Epoch 8/10
(===========================)Loss: 0.5527 Acc: 0.8519
=======)Loss: 0.6463 Acc: 0.7963
Epoch 9/10
(===========================)Loss: 0.5301 Acc: 0.8495
=======)Loss: 0.5559 Acc: 0.8380
Epoch 10/10
(===========================)Loss: 0.4903 Acc: 0.8507
=======)Loss: 0.4903 Acc: 0.8426
1e-05
Epoch 1

In [25]:
net = Net(32).to(device)

loss_fn = nn.NLLLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

train_and_evaluate(net, optimizer, loss_fn, dataloaders, device, lr=0.001, num_epoch=100)

Epoch 1/100
(===========================)Loss: 1.6670 Acc: 0.3206
=======)Loss: 2.1736 Acc: 0.2176
Epoch 2/100
(===========================)Loss: 1.1971 Acc: 0.5579
=======)Loss: 1.2827 Acc: 0.5139
Epoch 3/100
(===========================)Loss: 1.0464 Acc: 0.6331
=======)Loss: 1.0325 Acc: 0.6620
Epoch 4/100
(===========================)Loss: 0.9250 Acc: 0.6748
=======)Loss: 0.8625 Acc: 0.6667
Epoch 5/100
(===========================)Loss: 0.7989 Acc: 0.7523
=======)Loss: 0.7900 Acc: 0.7361
Epoch 6/100
(===========================)Loss: 0.7285 Acc: 0.7662
=======)Loss: 0.7673 Acc: 0.7315
Epoch 7/100
(===========================)Loss: 0.6607 Acc: 0.8218
=======)Loss: 0.7904 Acc: 0.7269
Epoch 8/100
(===========================)Loss: 0.6133 Acc: 0.8171
=======)Loss: 0.6375 Acc: 0.8056
Epoch 9/100
(===========================)Loss: 0.5544 Acc: 0.8472
=======)Loss: 0.7006 Acc: 0.7546
Epoch 10/100
(===========================)Loss: 0.5133 Acc: 0.8438
=======)Loss: 0.5925 Acc: 0.8194
Epoch 11/